In [1]:
import pandas as pd
import shapefile
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import geopandas as gp
from shapely.geometry import Point,Polygon
import bs4

## 讀實價登錄資料
- 去掉沒用的column

In [3]:
df1 = pd.read_excel('/projectdata/nckugeo01/實價登錄資料.xls')
df1 = df1.drop('Count IF',axis=1)

## Feature
- total_layers: 總樓層數
- Rezone_5: 標的是否在重劃區五期內
- Rezone_7: 標的是否在重劃區七期內
- 行政區：中區、北區、北屯區、南區、南屯區、后里區、外埔區、大安區、大甲區、大肚區、大里區、大雅區、太平區、新社區、東勢區、東區、梧棲區、沙鹿區、清水區、潭子區、烏日區、石岡區、神岡區、西區、西屯區、豐原區、霧峰區、龍井區（one-hot）
- admin_area: 標的是否在 ['西區','西屯區','潭子區','南屯區','大雅區','北屯區','南區','北區','豐原區','大里區','太平區']內
- 住宅大樓
- 公寓
- 套房
- 華廈
- 透天厝
- others: 標的屬於 ['倉庫','其他','工廠','農舍','廠辦']
- 商用: 標的屬於 ['店面(店鋪)','辦公商業大樓']
- may_be_zero: 標的屬於 ['倉庫','辦公商業大樓','工廠','農舍'] 可能缺少每坪單價
- BM_B: 標的建材屬磚造
- BM_R: 標的建材屬鋼筋混凝土

### total_layers

In [4]:
def convertChineseDigitsToArabic (chinese_digits, encoding="utf-8"):
    chs_arabic_map = {u'零':0, u'一':1, u'二':2, u'三':3, u'四':4,u'五':5, u'六':6, u'七':7, u'八':8, u'九':9,u'十':10}
    result  = 0
    tmp     = 0
    hnd_mln = 0
    for count in range(len(chinese_digits)):
        curr_char  = chinese_digits[count]
        curr_digit = chs_arabic_map.get(curr_char, None)
        if curr_digit >= 10:
            tmp    = 1 if tmp == 0 else tmp
            result = result + curr_digit * tmp
            tmp    = 0
        # meet single digit
        elif curr_digit is not None:
            tmp = tmp * 10 + curr_digit
        else:
            return result
    result = result + tmp
    result = result + hnd_mln
    return result

In [5]:
layer = df1.總樓層數.str[:-1]

layer = layer.fillna('零')

layer = layer.replace('地下','五十')

layer = layer.replace('見其他登記事','六十')

layer_num = [convertChineseDigitsToArabic(i) for i in layer]

layer_num = [-1 if i == 50 else i for i in layer_num ]

layer_num = [5 if i == 60 else i for i in layer_num ]

ladf = pd.DataFrame(layer_num,columns=['total_layers'])

### Rezone_5、7

In [10]:
Rezone = gp.GeoDataFrame.from_file('at071-group01/Rezone/Rezone.shp')

In [11]:
Rdf = pd.DataFrame()
for i in Rezone.Name:
    Rdf[i] = 0

for j in tqdm(range(len(Rezone))):
    a = []
    pal = Rezone.geometry[j]
    for i in range(len(df1)):
        poi = Point(df1.iloc[i].Response_X,df1.iloc[i].Response_Y)
        if pal.contains(poi):
            a.append(1)
        else:
            a.append(0)
    Rdf[Rezone.Name[j]] = a   

HBox(children=(IntProgress(value=0, max=49), HTML(value='')))

In [15]:
Rdf = Rdf[['五期','七期']]

In [16]:
Rdf.columns = ['Rezone_5','Rezone_7']

### Admin_area

In [19]:
all_ad = pd.get_dummies(df1.鄉鎮市區)

In [32]:
ar = ['西區','西屯區','潭子區','南屯區','大雅區','北屯區','南區','北區','豐原區','大里區','太平區']

y=[]
for i in df1.鄉鎮市區:
    if i in ar:
        y.append(1)
    else:
        y.append(0)
all_ad['admin_area'] = y

In [23]:
# col_n = ['ad_C', 'ad_N', 'ad_NT', 'ad_S', 'ad_ST', 'ad_HL', 'ad_YP', 'ad_DA', 'ad_DJ', 'ad_DD',
#        'ad_DL', 'ad_DY', 'ad_TP', 'ad_SS', 'ad_DS', 'ad_E', 'ad_WC', 'ad_CL', 'ad_CT', 'ad_TG',
#        'ad_WZ', 'ad_CK', 'ad_CG', 'ad_W', 'ad_WT', 'ad_FY', 'ad_WF', 'ad_LG']

### Building_type

In [26]:
btdf = pd.get_dummies(df1.建物型態)

In [28]:
btdf['others'] = btdf['倉庫'] + btdf['其他'] + btdf['工廠'] + btdf['農舍'] + btdf['廠辦']
btdf['商用'] = btdf['店面(店鋪)'] + btdf['辦公商業大樓']
btdf['may_be_zero'] = btdf['倉庫'] + btdf['辦公商業大樓'] + btdf['工廠'] + btdf['農舍']

btdf = btdf.drop(['倉庫','其他','工廠','農舍','廠辦','店面(店鋪)','辦公商業大樓'],axis=1)
btdf.columns = ['住宅大樓', '公寓', '套房', '華廈', '透天厝','others', '商用', 'may_be_zero']

### Building_Materials
- 鋼筋混凝土
- 磚造

In [35]:
y = []
for i in df1.主要建材.str.contains('鋼筋混凝土'):
    if i:
        y.append(1)
    else:
        y.append(0)

In [36]:
x = []
for i in df1.主要建材.str.contains('磚造'):
    if i:
        x.append(1)
    else:
        x.append(0)

In [45]:
bmdf = pd.DataFrame([x,y]).transpose()

bmdf.columns = ['BM_B','BM_R']

## Merge_dataframe

In [51]:
yoga_feature = pd.concat([ladf,Rdf,all_ad,btdf,bmdf],axis=1)

In [53]:
yoga_feature.to_csv('feature_by_yoga.csv',index=False)

## 去除特殊關係交易

In [ ]:
import pandas as pd

df = pd.read_csv('all_data.csv') #30000 筆 feature

df1 = pd.read_csv('at071-group01/tgos_80k.csv') #實價登陸的資料
df1 = df1[:30000]

df2 = pd.concat([df1,df],axis = 1)

df2['備註'] = df2.備註.fillna('無備註')

df2 = df2[~df2.備註.str.contains('間之')]

#然後再把不要的欄位清掉，我的做法是把 column 重新 insert 一次 (確定沒有重複的 Column name) 再 drop
#[i for i in df2.columns] #看有哪些欄位

## 內環、中環、外環

In [2]:
df = pd.read_csv('final_data_26528.csv')

In [4]:
df = df.drop('Unnamed: 0',axis=1)

In [ ]:
#把鄉鎮市區這個欄位復原
ad = []
for i in tqdm(range(len(df))):
    for j in df.columns[9:38]:
        if df[j][i] == 1:
            ad.append(j)

df['鄉鎮市區'] = ad

In [23]:
egg = df.groupby('鄉鎮市區')['單價(元/平方公尺)'].median()

In [21]:
en = egg.index
ev = egg.values

egg_dic = dict(zip(en,ev))

In [39]:
egg_out = [] #外環
egg_in = [] #內環
egg_mid = [] #中環
for i in en:
    if egg_dic[i] < 50000:
        egg_out.append(i)
    elif egg_dic[i] >= 70000:
        egg_in.append(i)
    else:
        egg_mid.append(i)

In [45]:
eo=[]
em=[]
ei=[]
for i in df['鄉鎮市區']:
    if i in egg_out:
        eo.append(1)
    else:
        eo.append(0)
    if i in egg_in:
        ei.append(1)
    else:
        ei.append(0)
    if i in egg_mid:
        em.append(1)
    else:
        em.append(0)

In [49]:
df['egg_out'] = eo
df['egg_in'] = ei
df['egg_mid'] = em

In [50]:
df = df.drop('鄉鎮市區',axis=1)

In [51]:
df.to_csv('final_plus_egg.csv', index = False)